In [1]:
from multiprocessing import Process, Queue

import numpy as np
from scipy.spatial.distance import cdist, pdist, squareform
from tqdm import tqdm_notebook

from tools.data_loading import load_images, load_labels, dummy_code, load_images_resized
from tools.feature_learning import pins_generation
from tools.hog import hog
from tools.kernels import kernel_matrix
from tools.optimization import find_f
from tools.prediction import pred
from tools.process_images import process_images
from tools.quantization import kmeans
from tools.submission import labels_to_csv
from tools.visualization import imshow, dump_as_png, reshape_as_images

# Data loading

In [2]:
X_train = load_images(type="train63")
n_train = X_train.shape[0]
Y_labels_train = load_labels()
Y_train = dummy_code(Y_labels_train)
n_classes = Y_train.shape[1]
X_train = X_train - X_train.min(axis=0)
X_train = X_train / X_train.max(axis=0)

In [3]:
image_list = reshape_as_images(X_train)

In [4]:
len(image_list)

5000

# Visual features

In [5]:
filter_sigma=0.1
filter_shape=5
hog_cell_size=8
disc_grid=16

In [6]:
n_images = image_list.shape[0]

In [7]:
hog_list = []
for i in tqdm_notebook(range(n_images), desc="Images"):
    image = image_list[i,:,:,:]
    hog_list.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid,
            color_grad=True))
    
n_features = hog_list[0].size
X_hog = np.array(hog_list).reshape((n_images, n_features)) 


# Data separation

In [8]:
training_idx = []
test_idx = []
train_frac = 0.9
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    selected_indices = np.random.permutation(np.sum(Y_labels_train == dig))
    n_class = len(selected_indices)
    class_indices = np.where(Y_labels_train == dig)[0]
    training_idx += list(map(int, class_indices[selected_indices[:int(train_frac*n_class)]]))
    test_idx += list(map(int,class_indices[selected_indices[int(train_frac*n_class):]]))

In [9]:
len(test_idx)

500

In [10]:
X_sample = X_hog[training_idx, :]
n_sample = X_sample.shape[0]
Y_sample = Y_train[training_idx, :]
Y_labels_sample = Y_labels_train[training_idx]

X_test = X_hog[test_idx, :]
n_test = X_test.shape[0]
Y_labels_test = Y_labels_train[test_idx]

## Pre-computing distances

In [11]:
train_dist = squareform(pdist(X_sample, "euclidean"))
train_test_dist = cdist(X_sample[:500], X_sample, "euclidean")
test_dist = cdist(X_test, X_sample, "euclidean")

## Pre-computing inner products

In [12]:
train_inner = X_sample.dot(X_sample.T)
train_test_inner = X_sample[:500].dot(X_sample.T)
test_inner = X_test.dot(X_sample.T)

# Training

## Classifier choice

In [13]:
classifier_type = "svm"
n_iter=60000

## Kernel choice

In [14]:
kernel_type = "polynomial"
degree = 4
constant = 5
lamb = 1

In [15]:
kernel_type="rbf"
sigma = 20
lamb = 1e-8

## Kernel computation

In [16]:
K_sample = kernel_matrix(
    X_sample,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=train_dist,
    inner=train_inner)

In [17]:
K_sample

array([[ 1.        ,  1.        ,  1.        , ...,  0.04880177,
         0.12252744,  0.11104589],
       [ 1.        ,  1.        ,  1.        , ...,  0.04880177,
         0.12252744,  0.11104589],
       [ 1.        ,  1.        ,  1.        , ...,  0.04880177,
         0.12252744,  0.11104589],
       ..., 
       [ 0.04880177,  0.04880177,  0.04880177, ...,  1.        ,
         0.14080545,  0.06737063],
       [ 0.12252744,  0.12252744,  0.12252744, ...,  0.14080545,
         1.        ,  0.13881935],
       [ 0.11104589,  0.11104589,  0.11104589, ...,  0.06737063,
         0.13881935,  1.        ]])

## Optimization

In [18]:
alpha = np.zeros((n_classes, n_sample))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    # weights = np.ones(n_sample)
    # weights[Y_labels_sample == dig] *= 10
    alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                           prob_type=classifier_type, lamb=lamb, n_iter=n_iter)

In [19]:
for i in alpha[1]:
    print(i)

0.0
0.0
0.0
0.0
0.0
0.0
-0.00565415104136
0.0
0.0
0.0
-0.0493752949931
-0.00163760444053
0.0
0.0
0.0
0.0
-0.00291396886624
0.0
-0.00282568622462
-0.000180168873347
0.0
0.0
-0.0020403738868
0.0
0.0
0.0
0.0
-0.0065749857959
0.0
0.0
0.0
0.0
-0.00121281053756
-0.00335587165762
0.0
-4.4408920985e-15
-0.00194242405205
0.0
0.0
-0.000739029343221
0.0
0.0
0.0
0.0
-0.00860944988526
-0.00509629078725
0.0
0.0
0.0
-0.0233397376967
0.0
0.0
0.0
-0.00626976415374
0.0
0.0
0.0
0.0
0.0
0.0
-0.000884318840032
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.000109676066272
-0.00222900625525
-0.0162460612384
-0.00315130243368
0.0
-4.4408920985e-16
0.0
-0.00192821813082
0.0
0.0
-0.000686612583865
0.0
-0.00232224454338
-0.00123960255672
0.0
-9.97128733089e-05
0.0
0.0
-0.000274902971583
-0.00425275256632
-0.00706547295005
0.0
0.0
-0.00231642468827
0.0
0.0
-0.00633667060564
-0.00936532870094
-0.00338127469746
0.0
0.0
-0.00192331145341
0.0
0.0
0.0
0.0
0.0
-0.0111809780264
-0.00074214456576
0.0
0.0
0.0
0.0
0.0
0.0
-4.4408920985e-

In [20]:
alpha[0].min()

-0.61513781109223953

# Evaluation

### Training error

In [21]:
Y_pred_train = pred(
    X_sample, X_sample[:500], alpha,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=train_test_dist,
    inner=train_test_inner)


Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
prec = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
print("The precision on the train set is of {}".format(prec))

The precision on the train set is of 1.0


### Testing error

In [29]:
Y_pred = pred(
    X_sample, X_test, alpha,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma)


Y_labels_pred = np.argmax(Y_pred, axis=1)
prec = np.mean(Y_labels_pred == Y_labels_test)
print("The precision on the test set is of {}".format(prec))

The precision on the test set is of 0.65


In [42]:
for i in Y_pred[:, 0]:
    print(i)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.63184560233
-1.37323066462
-1.55684292919
-1.04473261136
-1.5268689665
-1.44441008903
-1.33839623357
-1.475003285
-1.31812770773
-1.53049792374
-1.37920788283
-1.51142943648
-1.52716900889
-1.5186385069
-1.34892477015
-1.30113231603
-1.71211405542
-1.46789656839
-1.41699336324
-1.50569379979
-1.31540697186
-1.68449305344
-1.0596330288
-0.732400087788
-1.06746393379
-1.09599632552
-1.47793292021
-1.11570262865
-1.45191592031
-1.3682079192
-1.46553656193
-1.54878555372
-1.38355689676
-1.18554558943
-1.21638185526
-1.07971535732
-1.45471882379
-1.7712056136
-1.59925313574
-1.36202342319
-1.45207079573
-1.31526413548
-1.35838552739
-1.08132189221
-1.40749646855
-1.67747816091
-1.4855433621
-1.65508326089
-1.62867856125
-1.55530330863
-0.992037065435
-1.89004540774
-1.46241356174
-1.40301202

In [54]:
np.sum(Y_labels_pred==2)

35

## Grid search from hell

In [ ]:
def coordinate_ascent(out_q, dig, **kwargs):
    out_q.put(
        [
            dig,
            find_f(**kwargs)            
        ]
    )

### For polynomial

In [ ]:
kernel_type = "polynomial"
degrees = [4]
n_degrees = len(degrees)
constants = [1]
n_constants = len(constants)
lambs = [1]
n_lambs = len(lambs)
weights = [5, 10, 50, 1000]
n_weights = len(weights)
train_prec = np.zeros((n_degrees, n_constants, n_lambs, n_weights))
test_prec = np.zeros((n_degrees, n_constants, n_lambs, n_weights))
for d in tqdm_notebook(range(n_degrees), desc="Degrees"):
    for c in tqdm_notebook(range(n_constants), desc="Constants"):
        # Kernel matrix computation
        K_sample = kernel_matrix(
            X_sample, kernel_type=kernel_type, degree=degrees[d], constant=constants[c], inner=train_inner)
        for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
            for w in tqdm_notebook(range(n_weights), desc="Weights"):
                # Coordinate ascent with multiprocessing
                alpha = np.zeros((n_classes, n_sample))
                out_q = Queue()
                procs = list()
                for dig in range(n_classes):
                    weights = np.ones(n_sample)
                    weights[Y_labels_sample == dig] *= weights[w]
                    p = Process(
                        target=coordinate_ascent,
                        args=(out_q, dig),
                        kwargs={
                            "K":K_sample,
                            "Y":Y_sample[:, dig],
                            "weights": weights,
                            "prob_type":classifier_type, "lamb":lambs[l], "n_iter":100000
                        }
                    )
                    procs.append(p)
                    p.start()
                for i in range(n_classes):
                    res = out_q.get()
                    alpha[res[0], :] = res[1]

                for p in procs:
                    p.join()
                # Precision on train set
                Y_pred_train = pred(
                    X_sample, X_sample[:500], alpha,
                    kernel_type=kernel_type,
                    degree=degrees[d],
                    constant=constants[c],
                    inner=train_test_inner)

                Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
                train_prec[d, c, l, w] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
                # Precision on test set
                Y_pred= pred(
                    X_sample, X_test, alpha,
                    kernel_type=kernel_type,
                    degree=degrees[d],
                    constant=constants[c],
                    inner=test_inner)

                Y_labels_pred = np.argmax(Y_pred, axis=1)
                test_prec[d, c, l, w] = np.mean(Y_labels_pred == Y_labels_test)

In [ ]:
test_prec

In [ ]:
train_prec

### For RBF

In [ ]:
kernel_type="rbf"
sigmas = [20]
n_sigmas = len(sigmas)
weights = [10]
n_weights = len(weights)
lambs = [1e-8]
n_lambs = len(lambs)
train_prec_rbf = np.zeros((n_sigmas, n_lambs, n_weights))
test_prec_rbf = np.zeros((n_sigmas, n_lambs, n_weights))
for s in tqdm_notebook(range(n_sigmas), desc="Sigmas"):
    # Kernel matrix computation
    K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, sigma=sigmas[s], dist=train_dist)
    for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
        for w in tqdm_notebook(range(n_weights), desc="Weights"):
            # Coordinate ascent with multiprocessing
            alpha = np.zeros((n_classes, n_sample))
            out_q = Queue()
            procs = list()
            for dig in range(n_classes):
                weights = np.ones(n_sample)
                weights[Y_labels_sample == dig] *= weights[w]
                p = Process(
                    target=coordinate_ascent,
                    args=(out_q, dig),
                    kwargs={
                        "K":K_sample,
                        "Y":Y_sample[:, dig],
                        "weights": weights,
                        "prob_type":classifier_type, "lamb":lambs[l], "n_iter":40000
                    }
                )
                procs.append(p)
                p.start()
            for i in range(n_classes):
                res = out_q.get()
                alpha[res[0], :] = res[1]

            for p in procs:
                p.join()
            # Precision on train set
            Y_pred_train= pred(X_sample, X_sample[:500], alpha,
                                  kernel_type=kernel_type, sigma=sigmas[s], dist=train_test_dist)
            Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
            train_prec_rbf[s, l, w] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
            # Precision on test set
            Y_pred = pred(X_sample, X_test, alpha,
                                  kernel_type=kernel_type, sigma=sigmas[s], dist=test_dist)


            Y_labels_pred = np.argmax(Y_pred, axis=1)
            test_prec_rbf[s, l, w] = np.mean(Y_labels_pred == Y_labels_test)

In [ ]:
test_prec_rbf

In [ ]:
train_prec_rbf

# Prediction

In [24]:
print("filter sigma: {}".format(filter_sigma))
print("filter shape: {}".format(filter_shape))
print("hog cell size: {}".format(hog_cell_size))
print("discretization grid: {}".format(disc_grid))

filter sigma: 0.1
filter shape: 5
hog cell size: 8
discretization grid: 16


In [25]:
X_eval = load_images(type="test63")
n_eval = X_eval.shape[0]
X_eval = X_eval - X_eval.min(axis=0)
X_eval = X_eval / X_eval.max(axis=0)

image_list_eval = reshape_as_images(X_eval)

# Visual features for submission
hog_list_eval = []
for i in tqdm_notebook(range(n_eval), desc="Submission images"):
    image = image_list_eval[i,:,:,:]
    hog_list_eval.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid,
            color_grad=True))
    
n_features = hog_list_eval[0].size
X_hog_eval = np.array(hog_list_eval).reshape((n_eval, n_features)) 

In [26]:
print("Kernel type: {}".format(kernel_type))
print("Degree: {}".format(degree))
print("Sigma: {}".format(sigma))

Kernel type: rbf
Degree: 4
Sigma: 20


In [32]:
sub_dist = cdist(X_hog_eval, X_sample, "euclidean")

In [34]:
sub_dist

array([[ 37.75191792,  37.75191792,  37.75191792, ...,  43.52526227,
         33.05333426,  36.01603861],
       [ 49.4052441 ,  49.4052441 ,  49.4052441 , ...,  46.88066219,
         43.34775799,  46.25253696],
       [ 43.9841999 ,  43.9841999 ,  43.9841999 , ...,  43.08501886,
         33.10032494,  40.2720192 ],
       ..., 
       [ 46.7318359 ,  46.7318359 ,  46.7318359 , ...,  47.6348594 ,
         40.1532291 ,  42.22354568],
       [ 40.15805596,  40.15805596,  40.15805596, ...,  42.63803913,
         35.09491809,  36.50033705],
       [ 37.57539583,  37.57539583,  37.57539583, ...,  43.12716423,
         35.99937682,  39.88882176]])

In [35]:
Y_eval = pred(X_sample, X_hog_eval, alpha,
                      kernel_type=kernel_type, degree=degree, constant=constant, sigma=sigma, dist=sub_dist)


Y_labels_eval = np.argmax(Y_eval, axis=1)


# Submission
labels_to_csv(Y_labels_eval, kernel=kernel_type, algo="svm_hog",user="Pierre_color_grad_big2")

In [36]:
np.sum(Y_labels_eval==0)

0

In [38]:
for i in Y_eval[:, 1]:
    print(i)

-1.43304800614
-1.38594406261
-2.23232398186
-0.752479679953
-0.875590596863
-0.917247677491
-0.118270698367
0.0367676669053
-1.1583778815
-1.34748185395
-0.437089074422
-1.47555429306
-1.65465991624
-1.91414386883
-0.496970057178
-1.48449319346
-2.40679749672
-1.84578861756
-2.21500730706
-1.06995326239
-1.02756658765
-0.929850908667
-1.19051164827
-1.43064780075
-0.730548902533
-1.96382294663
-1.26466483972
-1.32145285461
-1.53891500165
-0.623583201868
-0.846618543011
-1.34477377642
-1.10061006147
-1.10101954661
-1.18432343052
-1.12230747048
-1.68260752347
-0.719161676011
-0.609164520742
-1.2023658908
-0.462519155909
1.08412812272
-0.122340887758
-0.754566360174
-1.74538980187
-1.11308235182
-1.51673587381
-0.45593431442
-0.971289347266
-1.29140576519
-1.27395598447
-0.552116158041
-1.03474746681
-1.68648315734
-0.331693603948
0.574881161179
-1.25968877628
-0.760919636238
-0.586575006692
-1.01120182113
-0.850881589796
-2.15087853032
0.734705524202
-0.999958876391
-1.17621641755
-0.49

In [41]:
Y_eval[21]

array([-1.50887173, -0.92985091, -1.83514134, -1.34510263, -0.36576481,
       -1.48228314, -0.78744167, -0.72426505, -1.36872561,  0.50707252])